In [ ]:
def fun_c_opt(delay, vct_h, pow_noise):

    ITER = SYMBOLS
    vct_ak_th = np.zeros(((N + len(vct_h) + delay), 1), dtype=np.complex_) 
    Ak = vct_ak_th[delay]
    vct_rk_th = np.zeros((N, 1), dtype=np.complex_)     
    vct_alpha_it = np.zeros((N, 1, ITER), dtype=np.complex_)
    matrix_phi_it = np.zeros((N, N, ITER), dtype=np.complex_)

    for it_mc in range(1, ITER  +1):

      vct_ak_th[1:] = vct_ak_th[0:-1]
      vct_ak_th[0] = (np.random.randint(0,1 +1) + np.random.randint(0,1 +1)*1j)*2-1 # Complex random numbers generation
      vct_ak_for_conv = vct_ak_th[0:N]
      Ak = vct_ak_th[delay] 
      
      awgn = np.sqrt(0.5 * pow_noise) * (np.random.randn() + 1j * np.random.randn()) # the sqrt(0.5) because it is complex

      vct_rk_th[1:] = vct_rk_th[0:-1] 
      vct_rk_th[0] = vct_h.T @ vct_ak_for_conv + awgn

      vct_alpha_it[:, :, it_mc -1] = Ak * np.conj(vct_rk_th) # eq. 9.9
      matrix_phi_it[:, :, it_mc -1] = np.conj(vct_rk_th) @ vct_rk_th.T

    vct_alpha = vct_alpha_it.sum(axis=2)/ITER
    matrix_phi = matrix_phi_it.sum(axis=2)/ITER # The Toeplitz matrices are added in order to calculate average Hermitian Toeplitz matrix.
    inv_matrix_phi = np.linalg.pinv(matrix_phi) # Moore-Penrose pseudo inverse, to avoid singular matrix error when vectors has two equal elements
    vct_c_opt = (inv_matrix_phi @ vct_alpha).T[0] # eq. 9.16
      
    return [vct_c_opt, vct_alpha, matrix_phi]